# *Init*

In [1]:
#@title Imports

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
# #@title Mount Data (For Google Colab)
# # Mount data drive

# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
# %cd /content/drive/MyDrive/CPSC4300-ADS-Project

# **Unhealthy Tree Detection in Segmented Drone Footage via Machine Learning**
**Clemson University | Fall 2023**<br>
**Authors:** Scott Logan, Lisa Umatoni, Mostafa Saberian, Ian McCall, Neil Kuehn


NOTES:

Look into adapting a pretrained model, such as VGG or Resnet, and retraining only the last few layers.

# **Project Goals!**

## The Mission
In this project, our goal is to analyze segmented drone footage to locate and annotate unhealthy trees found within the images using a Machine Learning model.


### To Do
- Finish Data Cleaning Methods
- Implement CNNs 2,3
- Try implementing 4: Look into adapting a pretrained model, such as VGG or Resnet, and retraining only the last few layers.

# **Data Summary**

The provided data set is a total of 60 images of trees, with 37 healthy trees and 23 sick tree images given. The main unit of analysis for determining whether a tree is sick or not is color. Healthy trees are greener and darker, whereas sick trees are yellowed and lighter. 

Note that dead areas, such as bare tree branches, are not counted as sick.

# **Data Cleaning Strategies**

In a photograph, there is plenty of noise that may distract a Machine Learning Model from the data that is being represented. As such, it may be beneficial to perform data cleaning steps before training or predicting using the model.


We have decided to implement the following data cleaning steps, and test the model's performance using various combinations of these steps:

**Greyscale:** Normalize each pixel into a single grey value

**Isolate Hue:** Isolate the Red channel of each pixel
> Tree health is largely defined by yellowing, which in an RGB value is defined by an increase in the Red value. As such, we may be interested primarily in the Red channel, and may increase model accuracy by isolating or at least exaggerating the Red channel of images during processing.

**Omit Values Beyond Range of Interest:** Remove information likely to confuse the model
> Areas of images with red values that are too high are likely to be unrelated to tree data, so they should be omitted


In [3]:
#@title Greyscale
def greyscale(input_img):
    output_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    return output_img

In [4]:
#@title Normalize Saturation and Value:
def normalize_saturation_value(input_img):
    # Convert the image from BGR to HSV color space
    hsv_image = image.rgb_to_hsv(input_img)
    #hsv_image = cv2.cvtColor(input_img, cv2.COLOR_BGR2HSV)

    # Split the HSV image into separate channels
    h, s, v = cv2.split(hsv_image)

    # Normalize the saturation and value channels
    s = np.uint8(np.clip((s * 1.2), 0, 255))
    v = np.uint8(np.clip((v * 1.2), 0, 255))

    # Merge the normalized channels back into an HSV image
    hsv_image = cv2.merge([h, s, v])

    # Convert the HSV image back to BGR color space
    output_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    return output_image

In [5]:
#@title Isolate Hue:
def isolate_hue(input_img):
    
    # Exaggerate the red channel by decreasing blue and green by 80%
    output_img = input_img[:, :, :2] * 0.2

    return output_img

In [6]:
#@title Omit Unwanted Pixel Data Beyond Range of Interest
width = 225
height = 400
threshold_excess_red = 20
threshold_green_lower_bound = 20

def omit_unwanted_ranges(input_img,img_width = width, img_height = height):
    img_pixels = input_img.load()
    for i in range(img_height):
        for j in range(img_width):
            r,g,b = img_pixels[i,j]
            if (g < threshold_green_lower_bound):
                r = 0
                g = 0
            if (r > threshold_excess_red):
                r = 0
                g=0
            b = 0
            
    
    output_img = input_img
    return output_img

In [7]:
#@title Resize Images

def resize_img(input_img, targetwidth, targetheight):
    output_img = tf.image.resize(input_img, [targetwidth, targetheight])
    return output_img

# **Initial Model:** Convolutional Neural Network

We considered a number of different models for this project, choosing CNN as our initial model selection:
- **-> Convolutional Neural Network (CNN):** Suitable for classifying photos by visible features, which we plan to use by training the CNN to detect color patterns typical of sick trees.
- **Clustering Model:** May be useful to detect multiple instances of sick trees within an image, using elbow method to determine number of sick tree instances.
- **Classification Model:** Suitable for binary response values, which may be useful to classify healthy vs. sick.



## Model Implementations

A few different CNN implementations will be tested, primarily as an exploration of how CNNs work.

- Implementation 1: Brute-Force Classification Approach


- Implementation 2: 


- Implementation 3?: Repurpose trained model

### Implementation 1: Brute-Force Classification Approach

In our first implementation, the Brute-Force Classification Approach, all we will be doing is passing the healthy images and the sick images into a CNN and training it to categorize the images into healthy and sick. This approach is primarily to ensure that we can construct a functioning CNN model as a baseline before attempting more involved strategies.

- Training Data:
  - Cleaning Steps: Resize Images
- Model Architecture
  - Convolutional Layers: 
  - Two Pooling Layers: 
  - Two Dense Layers:
    - 128-unit Relu layer: Learn non-linear transformations of features to capture complex relationships between features.
    - 1-unit Sigmoid layer: Filters output into a single binary value: 0 for healthy (lacks sick features), 1 for sick (contains sick features)

In [15]:
# Define Data Cleaning step for Model
default_width, default_height = 225, 225
default_size = (default_width, default_height)
def clean_data(input_img, resize_width=default_width, resize_height=default_height):
    clean_img = input_img
    
    # Apply Data Cleaning Functions
    clean_img = resize_img(clean_img, resize_width, resize_height)
    
    output_img = clean_img
    return output_img

In [16]:
# Define Data Augmentation step for model
# Apply a number of random transformations on data set to increase the model's accuracy on more general data.
# Chosen transformations: rotations, width-shift, height-shift, shear, zoom, horizontal flips
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=clean_data
)

In [29]:
# Use tensorflow's Data Generator to create training and validation sets
train_generator = train_datagen.flow_from_directory(
    'data/data_model_0',
    target_size=(default_width, default_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'data/data_model_0',
    target_size=(default_width, default_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Build model with chosen layers
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Found 49 images belonging to 2 classes.
Found 11 images belonging to 2 classes.
Epoch 1/10
2/2 [==============================] - 10s 4s/step - loss: 6.4979 - accuracy: 0.6122 - val_loss: 1.7770 - val_accuracy: 0.6364
Epoch 2/10
2/2 [==============================] - 8s 3s/step - loss: 3.9051 - accuracy: 0.5102 - val_loss: 2.0478 - val_accuracy: 0.3636
Epoch 3/10
2/2 [==============================] - 7s 5s/step - loss: 1.6152 - accuracy: 0.3878 - val_loss: 0.6539 - val_accuracy: 0.6364
Epoch 4/10
2/2 [==============================] - 7s 3s/step - loss: 0.6753 - accuracy: 0.6122 - val_loss: 0.6283 - val_accuracy: 0.6364
Epoch 5/10
2/2 [==============================] - 8s 3s/step - loss: 0.6537 - accuracy: 0.6122 - val_loss: 0.6305 - val_accuracy: 0.6364
Epoch 6/10
2/2 [==============================] - 8s 3s/step - loss: 0.6340 - accuracy: 0.6122 - val_loss: 0.6152 - val_accuracy: 0.6364
Epoch 7/10
2/2 [==============================] - 8s 5s/step - loss: 0.6420 - accuracy: 0.6122 - 

#### **Results and Discussion:** Brute-Force Classification Approach

In [31]:
# Get the number of validation steps
validation_steps = validation_generator.samples / validation_generator.batch_size

# Evaluate the model on the validation data
loss, accuracy = model.evaluate(validation_generator)

print(f"Validation loss: {loss}")
print(f"Validation accuracy: {accuracy}")

1/1 [==============================] - 1s 1s/step - loss: 0.6504 - accuracy: 0.6364
Validation loss: 0.6504026651382446
Validation accuracy: 0.6363636255264282


With an end validation loss of 0.620 and a validation accuracy of 63.6%, the model appears to be fairly effective at classifying the images into healthy vs. sick.

However, I don't have much confidence in a result this good for the **Brute-Force** Classification Approach. During training, the model's accuracy was high from the start and remained about equally this high throughout training. To me, this is highly suspicious.

In [32]:
train_loss = history.history['loss']
train_accuracy = history.history['accuracy']
validation_loss = history.history['val_loss']
validation_accuracy = history.history['val_accuracy']

# Plot Loss and Accuracy throughout training
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(train_loss, label='Training Loss')
ax1.plot(validation_loss, label='Validation Loss')
ax1.set_title('Loss vs. Epochs')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(train_accuracy, label='Training Accuracy')
ax2.plot(validation_accuracy, label='Validation Accuracy')
ax2.set_title('Accuracy vs. Epochs')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()

# Show the plots
plt.tight_layout()
plt.show()

I suspect that due to the limited sample size of our data (only 60 images is not much), there are trends in the images that correlate to the healthy vs sick image samples that are unrelated to the sick features of the trees. For example, visually inspecting the data shows that many of the healthy tree images contain the sky, and none of the sick images do. The healthy tree images also happen to be on average more exposed than sick images.

Because this is a classification approach, a visualization technique other than image labeling is not possible. As such, we'll need to modify our model to classify images as well as locate the sick features.

### Implementation 2: 